# **EXPERIMENT 1**

The dataset used here is Emotions dataset for NLP. Each sentence from this dataset has one emotion label associated with it (e.g. "I am sad" -> sadness).

## Import packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import tensorflow as tf
from IPython.display import display
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from keras.layers import Embedding, Bidirectional, LSTM, GRU, Dense
from keras.models import Sequential

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Download and preprocess data

In [ ]:
with open('../data/emotions-dataset-for-nlp/train.txt', 'r') as f:
    X_train = []
    y_train = []
    for line in f:
        text, emotion = line.strip().split(';')
        X_train.append(text)
        y_train.append(emotion)

with open('../data/emotions-dataset-for-nlp/test.txt', 'r') as f:
    X_test = []
    y_test = []
    for line in f:
        text, emotion = line.strip().split(';')
        X_test.append(text)
        y_test.append(emotion)

with open('../data/emotions-dataset-for-nlp/val.txt', 'r') as f:
    X_test = []
    y_test = []
    for line in f:
        text, emotion = line.strip().split(';')
        X_test.append(text)
        y_test.append(emotion)

data_train = pd.DataFrame({'Text': X_train, 'Emotion': y_train})
data_test = pd.DataFrame({'Text': X_test, 'Emotion': y_test})
data = data_train.append(data_test, ignore_index=True)
display(data.head())


Text  Emotion
0                            i didnt feel humiliated  sadness
1  i can go from feeling so hopeless to so damned...  sadness
2   im grabbing a minute to post i feel greedy wrong    anger
3  i am ever feeling nostalgic about the fireplac...     love
4                               i am feeling grouchy    anger

In [ ]:
def clean_text(data):
    data = re.sub(r"(#[\d\w\.]+)", '', data)
    data = re.sub(r"(@[\d\w\.]+)", '', data)
    data = word_tokenize(data)
    return data

texts = [' '.join(clean_text(text)) for text in data.Text]
texts_train = [' '.join(clean_text(text)) for text in X_train]
texts_test = [' '.join(clean_text(text)) for text in X_test]

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequence_train = tokenizer.texts_to_sequences(texts_train)
sequence_test = tokenizer.texts_to_sequences(texts_test)
index_of_words = tokenizer.word_index
vocab_size = len(index_of_words) + 1
print(texts_train)
print(sequence_test)
print(index_of_words)

['i didnt feel humiliated', 'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake', 'im grabbing a minute to post i feel greedy wrong', 'i am ever feeling nostalgic about the fireplace i will know that it is still on the property', 'i am feeling grouchy', 'ive been feeling a little burdened lately wasnt sure why that was', 'ive been taking or milligrams or times recommended amount and ive fallen asleep a lot faster but i also feel like so funny', 'i feel as confused about life as a teenager or as jaded as a year old man', 'i have been with petronas for years i feel that petronas has performed well and made a huge profit', 'i feel romantic too', 'i feel like i have to make the suffering i m seeing mean something', 'i do feel that running is a divine experience and that i can expect to have some type of spiritual encounter', 'i think it s the easiest time of year to feel dissatisfied', 'i feel low energy i m just thirsty', 'i have i

In [ ]:
num_classes = 6
embed_num_dims = 300
max_seq_len = 500
class_names = ['anger', 'sadness', 'fear', 'joy', 'surprise', 'love']
X_train_pad = pad_sequences(sequence_train, maxlen=max_seq_len)
X_test_pad = pad_sequences(sequence_test, maxlen=max_seq_len)
encoding = {'anger': 0, 'sadness': 1, 'fear': 2, 'joy': 3, 'surprise': 4, 'love': 5}
y_train = [encoding[x] for x in data_train.Emotion]
y_test = [encoding[x] for x in data_test.Emotion]
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

## Word Embeddings

In [ ]:
if not os.path.exists('./word-vectors/'):
    !wget -P word-vectors/ https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
    !unzip word-vectors/wiki-news-300d-1M.vec.zip -d word-vectors/

--2021-12-16 00:42:50--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 681808098 (650M) [application/zip]
Saving to: ‘word-vectors/wiki-news-300d-1M.vec.zip’

wiki-news-300d-1M.v 100%[===================>] 650.22M  18.8MB/s    in 36s     

2021-12-16 00:43:26 (18.3 MB/s) - ‘word-vectors/wiki-news-300d-1M.vec.zip’ saved [681808098/681808098]

Archive:  word-vectors/wiki-news-300d-1M.vec.zip
  inflating: word-vectors/wiki-news-300d-1M.vec  


In [ ]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word]
                embedding_matrix[idx] = np.array(vector, dtype = np.float32)[:embedding_dim]
    return embedding_matrix
    
fname = 'word-vectors/wiki-news-300d-1M.vec'
embedd_matrix = create_embedding_matrix(fname, index_of_words, embed_num_dims)

## Model

In [ ]:
embedd_layer = Embedding(vocab_size, embed_num_dims, input_length=max_seq_len, weights=[embedd_matrix], trainable=False)
gru_output_size = 128
bidirectional = True
model = Sequential()
model.add(embedd_layer)
model.add(Bidirectional(GRU(units=gru_output_size, dropout=0.2 ,recurrent_dropout=0.2)))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## Training

In [ ]:
batch_size = 128
epochs = 8
hist = model.fit(X_train_pad, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test_pad, y_test))

Epoch 1/8
125/125 [==============================] - 909s 7s/step - loss: 1.3976 - accuracy: 0.4739 - val_loss: 1.1616 - val_accuracy: 0.5830
Epoch 2/8
125/125 [==============================] - 900s 7s/step - loss: 0.8880 - accuracy: 0.6918 - val_loss: 0.6566 - val_accuracy: 0.7635
Epoch 3/8
125/125 [==============================] - 910s 7s/step - loss: 0.5236 - accuracy: 0.8184 - val_loss: 0.3778 - val_accuracy: 0.8720
Epoch 4/8
125/125 [==============================] - 904s 7s/step - loss: 0.3424 - accuracy: 0.8777 - val_loss: 0.2449 - val_accuracy: 0.9150
Epoch 5/8
125/125 [==============================] - 895s 7s/step - loss: 0.2596 - accuracy: 0.9067 - val_loss: 0.2047 - val_accuracy: 0.9275
Epoch 6/8
125/125 [==============================] - 904s 7s/step - loss: 0.2160 - accuracy: 0.9207 - val_loss: 0.1724 - val_accuracy: 0.9310
Epoch 7/8
125/125 [==============================] - 900s 7s/step - loss: 0.1971 - accuracy: 0.9241 - val_loss: 0.1682 - val_accuracy: 0.9290
Epoch 

## Done

In [ ]:
message = ['I am mad.']
seq = tokenizer.texts_to_sequences(message)
padded = pad_sequences(seq, maxlen=max_seq_len)
pred = model.predict(padded)
print('Message:' + str(message))
print('Emotion:', class_names[np.argmax(pred)])

model.save('./exp-1/saved-model')

Message:['I am mad.']
Emotion: anger
INFO:tensorflow:Assets written to: ./drive/MyDrive/eecs595-final-project/exp-1/textmodel/assets


INFO:tensorflow:Assets written to: ./drive/MyDrive/eecs595-final-project/exp-1/saved-model/assets


INFO:tensorflow:Assets written to: ./drive/MyDrive/eecs595-final-project/exp-1/saved-model/assets


In [ ]:
loaded_model = tf.keras.models.load_model('./exp-1/saved-model')
loaded_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 300)          4858500   
                                                                 
 bidirectional_1 (Bidirectio  (None, 256)              330240    
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 6)                 1542      
                                                                 
Total params: 5,190,282
Trainable params: 331,782
Non-trainable params: 4,858,500
_________________________________________________________________
